In [5]:
rm(list = ls())
#setwd('/Users/zhengwei/Desktop/OneDrive - Texas A&M University-Corpus Christi/Research/Huiyan_Sang/DecisionTree/MyPackage/sim1_lin')
#setwd("/Users/zhengwei/OneDrive - Texas A&M University-Corpus Christi/Research/Huiyan_Sang/DecisionTree/MyPackage/sim1_lin")

library(foreach)
library(doMC)
library(coda)
library(MonBartSFM)
library(frontier)
library(truncnorm)
library(sn)
library(MASS)
library(semsfa)
source("otherfuns.R")
source("Bayes_SFM_NHN_Gibbs.R")

# Set path to your folder
path <- "simulated_data"

# List all CSV files in that folder
files <- list.files(path, pattern = "\\.csv$", full.names = TRUE)

# Read each CSV into a list of data.frames
simulated_datasets_R <- lapply(files, read.csv)


#set cores for parallel computing

part=1

  ##########################################
  #    simulate data for linear case       #
  ##########################################
  ## simulate simple data with one x
  beta0=log(2)
  beta1=0.2
  sigmav=0.9
  sigmau=0.2
  nsamp=200
i=1
  x_org=simulated_datasets_R[[i]]$log_X
  y_org=simulated_datasets_R[[i]]$log_y


  ########################################
  # Model 1 fit the Bayes Linear Model
  ########################################

  simdt=data.frame(y1=y_org,lnx1=x_org)

  B_burnin=1000
  B_afterburnin=4000

  # B_burnin=2
  # B_afterburnin=20

  res_Bayelin<-Bayeslinear_NHN_fit(B_burnin,
                                   B_afterburnin,
                                   simdt,
                                   beta0_init = 0,
                                   beta_init=beta0,
                                   sigu_init=sigmau,
                                   sigv_init=sigmav,
                                   df_v = 5,
                                   S_v =(5+2)*sigmav^2,
                                   df_u =5,
                                   S_u = (5+2)*sigmau^2)


  B=B_burnin+B_afterburnin
  res_bayes_lin<-na.omit(res_Bayelin$res[B_burnin+1:B,])

  ##############################################
  # Model 2
  # fit SEM-SFM
  # imposing monotonicity restrictions on inputs
  ##############################################
  #first-step: monotone gam, second-step: fan
  dati=data.frame(y=y_org,x=x_org)
  semfit<-semsfa(y~pbm(x,mono="up"),sem.method = "gam.mono",dati)
  sigma_v_semfit=sqrt(semfit$sigma^2/(1+semfit$lambda^2))
  sigma_u_semfit=semfit$lambda*sigma_v_semfit

  ##############################################
  # Model 3
  # fit Mon-BART-SFM
  # imposing monotonicity restrictions on inputs
  ##############################################
  #find initial value for sigma_v and sigma_u

  fit_sfa <-sfa( y1 ~ lnx1,data = simdt)
  gamma <- unname( coef(fit_sfa)["gamma"] )
  sigmaSq <- unname( coef(fit_sfa)["sigmaSq"] )
  sigmaSqU <- gamma * sigmaSq
  sigmaSqV <- ( 1 - gamma ) * sigmaSq
  betavec<-coef(fit_sfa)[1:3]
  sigma_u<-sqrt(sigmaSqU)
  sigma_v<-sqrt(sigmaSqV)



  ysnfit<-selm(y_org ~ 1, family="SN")
  center_Y <- ysnfit@param$dp[1]

  nskip=B_burnin
  ndpost=B_afterburnin


  fitbartsfm<-monbartsfm(
    x.train=as.matrix(x_org),y.train=y_org, x.test=matrix(0.0,0,0),
    sigest=sigma_v,
    sigdf=3,
    sigquant=.95,
    sigu_pre_est=sigma_u,
    sigquant_u=.95,
    k=2.0,
    power=.8, base=.25, #regular bart is 2,.95
    sigmaf=NA,
    lambda=NA,
    fmean = center_Y,
    #fscale=scale_Y,
    ntree=100,
    ndpost, nskip,
    mgsize=50,
    nkeeptrain=ndpost,nkeeptest=ndpost,
    nkeeptestmean=ndpost,
    nkeeptreedraws=ndpost,
    printevery=100
  )



  fitbartsfm_beta0<-monbartsfm_beta0(
    x.train=as.matrix(x_org),y.train=y_org, x.test=matrix(0.0,0,0),
    sigest=sigma_v,
    sigdf=3,
    sigquant=.95,
    sigu_pre_est=sigma_u,
    sigquant_u=.95,
    k=2.0,
    power=.8, base=.25, #regular bart is 2,.95
    sigmaf=NA,
    lambda=NA,
    fmean = center_Y,
    #fscale=scale_Y,
    ntree=100,
    ndpost, nskip,
    mgsize=50,
    nkeeptrain=ndpost,nkeeptest=ndpost,
    nkeeptestmean=ndpost,
    nkeeptreedraws=ndpost,
    printevery=100
  )


Warning message:
“package ‘foreach’ was built under R version 4.2.3”
Loading required package: iterators

Warning message:
“package ‘iterators’ was built under R version 4.2.3”
Loading required package: parallel

Loading required package: micEcon


If you have questions, suggestions, or comments regarding one of the 'micEcon' packages, please use a forum or 'tracker' at micEcon's R-Forge site:
https://r-forge.r-project.org/projects/micecon/

Loading required package: lmtest

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Please cite the 'frontier' package as:
Tim Coelli and Arne Henningsen (2013). frontier: Stochastic Frontier Analysis. R package version 1.1. http://CRAN.R-Project.org/package=frontier.

If you have questions, suggestions, or comments regarding the 'frontier' package, please use a forum or 'tracker' at frontier's R-Forge site:
https://r-forge.r-project.org/projects/frontie

Cumulative average: iter= 1000 beta0= 0.7941 beta= 0.2108 sigmav= 0.8698 sigmau= 0.255 
Cumulative average: iter= 2000 beta0= 0.7907 beta= 0.2099 sigmav= 0.87 sigmau= 0.2507 
Cumulative average: iter= 3000 beta0= 0.7944 beta= 0.2097 sigmav= 0.8689 sigmau= 0.2558 
Cumulative average: iter= 4000 beta0= 0.7902 beta= 0.2097 sigmav= 0.8687 sigmau= 0.2505 
Cumulative average: iter= 5000 beta0= 0.8042 beta= 0.2097 sigmav= 0.8658 sigmau= 0.2675 


Loading required package: Rcpp



This is my mon-Bart code
*****Into main of monotonic bart
**********************
n: 200
p: 1
first and last y: -0.781881, -0.183035
first row: 2.768552, 2.768552
second row: -2.856968, -2.856968
last row: 1.221842, 1.221842
no test observations
tau: 0.151553
nu: 3.000000
lambda: 0.067982
tree prior base: 0.250000
tree prior power: 0.800000
burn (nskip): 1000
nd (ndpost): 4000
m (ntree): 100
nm (mu grid size): 50
*****nkeeptrain,nkeeptest,nkeeptestme, nkeeptreedraws: 4000, 4000, 4000, 4000
*****printevery: 100
*****skiptr,skipte,skipteme,skiptreedraws: 1,1,1,1
**********************
dip.n: 0
x1 cuts: -2.934925 ... 2.914727
check counts
trcnt,tecnt,temecnt,treedrawscnt: 4000,0,0, 4000
This is my mon-Bart code
*****Into main of monotonic bart
**********************
n: 200
p: 1
first and last y: -0.781881, -0.183035
first row: 2.768552, 2.768552
second row: -2.856968, -2.856968
last row: 1.221842, 1.221842
no test observations
tau: 0.151553
nu: 3.000000
lambda: 0.067982
tree prior base: 0.

In [15]:
apply(fitbartsfm_beta0$yhat.train,2,median)

[1]  1.1594351913 -0.1064798429  0.4284138335  1.1059458236  0.4284138335
  [6]  0.3214350982  0.4462436227  0.5175627796  0.4640734119  0.9989670884
 [11]  0.8028394070  0.0183286816  0.8563287746  0.1609669953  0.4284138335
 [16]  0.9276479315  1.0881160344  0.8919883531  0.9989670884  0.8741585639
 [21]  0.4284138335  0.1966265737  0.0539882600  0.3036053090  0.8919883531
 [26]  1.1772649805  0.4284138335  1.1059458236  0.8741585639  1.1772649805
 [31]  0.7850096178  1.1772649805  0.4284138335  0.1609669953  0.0004988924
 [36]  0.2144563629  0.5175627796  1.1416054021  0.4462436227 -0.0173308969
 [41]  0.4819032011  0.4105840443  0.9276479315  0.7136904609  0.4284138335
 [46]  1.0881160344  0.6245415149  0.9276479315  0.0183286816  0.6602010933
 [51] -0.0173308969  0.9098181423  0.6067117256  0.8919883531 -0.0708202645
 [56]  0.3570946766  0.6245415149  0.4819032011  0.9633075099  1.1416054021
 [61]  0.9454777207  0.9276479315  0.8384989854  0.0361584708  0.3749244659
 [66]  0.3749244659  0.4640734119  0.9811372991  0.6067117256  0.8384989854
 [71]  0.9633075099  0.3392648874  0.9989670884  0.5353925688  0.5175627796
 [76]  0.8384989854  0.4997329904  0.8384989854 -0.1243096321  0.9633075099
 [81]  0.4997329904  0.6780308825  0.4819032011  0.2501159414  0.5175627796
 [86]  0.4997329904  0.4997329904  0.4284138335  0.8028394070  0.9633075099
 [91]  1.1416054021  0.4819032011  0.3392648874  0.0004988924  1.1416054021
 [96] -0.1243096321  1.1594351913  0.9989670884  0.9098181423  0.5353925688
[101]  0.5710521472  0.8919883531  0.4284138335  0.9098181423  0.4997329904
[106]  0.4462436227  0.5888819364  0.2501159414  0.9989670884  1.1772649805
[111]  1.1772649805  0.2322861521  0.4284138335  0.4819032011  0.3036053090
[116]  0.9098181423  0.0539882600  0.1609669953  0.4819032011  0.1609669953
[121]  0.8028394070  1.1237756129  0.4819032011  0.3927542551 -0.1064798429
[126]  0.8741585639  1.0346266668  0.8563287746 -0.0529904753  0.5353925688
[131]  0.4997329904  0.5710521472  0.0361584708  0.2501159414  0.4105840443
[136]  0.0361584708  0.9989670884  0.7671798286  1.0524564560  0.8741585639
[141] -0.0173308969  0.4284138335 -0.0708202645  0.3927542551  0.2857755198
[146] -0.1243096321  0.0183286816 -0.1243096321  0.0004988924  0.4819032011
[151]  0.9454777207  0.8919883531  0.8919883531  1.0346266668  0.3392648874
[156]  0.4997329904 -0.0886500537 -0.1064798429  0.3392648874  0.8919883531
[161]  0.9989670884  0.4640734119  0.6245415149  0.5353925688 -0.1243096321
[166]  0.8741585639  0.5175627796  1.1059458236  0.1609669953  0.8384989854
[171]  0.9098181423  0.4462436227  0.4640734119  0.7671798286 -0.0173308969
[176]  0.8384989854  0.9633075099  0.2322861521  0.9989670884  0.4462436227
[181]  0.4819032011  1.1594351913  0.3214350982  0.2857755198  0.2679457306
[186]  0.1609669953  0.8563287746  0.8028394070  0.2501159414  1.1594351913
[191]  0.4284138335  0.5175627796  1.0524564560  0.5532223580  0.8028394070
[196]  0.9989670884 -0.0708202645  0.9098181423  0.4284138335  0.9276479315